# Smoothing Test

In [1]:
from sogaPreprocessor import *
from producecfg import *
from libSOGA import *

torch.set_default_dtype(torch.float64)

In [ ]:
SMOOTH_EPS = 1e-2
SMOOTH_DELTA = 1e-2

def check_dist_non_deg(dist):    
    """ Checks whether all cov matrices in dist are non-degenerate."""
    sigma = dist.gm.sigma
    dets = torch.linalg.det(sigma)
    if torch.any(torch.abs(dets) < 1e-10):
        return True
    else:
        return False
    
def smooth_asgmt(dist, node, data, params_dict):
    """ dist is a distribution that when updated with node.expre is degenerate.
    This function adds to node.expr a Gaussian noise to avoid degeneracy and computes the new dist.
    The new expression for the update is saved in the attribute node.smooth"""
    current_EPS = SMOOTH_EPS
    new_expr = None
    smooth_flag = True
    while smooth_flag:
        new_expr = node.expr + '+ gm([1.], [0.], [{}])'.format(current_EPS)
        updated_dist = update_rule(dist, new_expr, data, params_dict)
        smooth_flag = check_dist_non_deg(updated_dist)
        current_EPS += SMOOTH_DELTA
    node.smooth = new_expr
    return updated_dist

def smooth_trunc(trunc, node):
    """ trunc is a truncation var ('==' | '!=') val.
    This functions transforms the conditions:
    var == val -> var > val - delta and var < val + delta
    var != val -> var < val - delta or var > val + delta
    where delta is selected by the function select_delta, depending on the current distribution"""
    if '==' in trunc:
        ops = '=='
    elif '!=' in trunc:
        ops = '!='
    target_var, target_val = trunc.split(ops)
    target_val = eval(target_val)
    dist = node.dist 
    delta = select_delta(dist, target_var, target_val)
    if ops == '==':
        new_trunc = '{} > {} and {} < {}'.format(target_var, target_val - delta, target_var, target_val + delta)
    elif ops == '!=':
        new_trunc = '{} < {} or {} > {}'.format(target_var, target_val - delta, target_var, target_val + delta)
    node.smooth = new_trunc
    return new_trunc    

def select_delta(dist, var, val):
    """ Selects the delta for the smooth_trunc function.
    It is based on the standard deviation of the variable var in the distribution dist."""
    var_idx = dist.var_list.index(var)
    # computes the component of the mixturewith mean closest to val
    marg_mean = dist.gm.mu[:,var_idx]
    diff = torch.abs(marg_mean - val)
    min_index = torch.argmin(diff)   # torch.argsort for ranked indices
    # selects the std of the component
    std = torch.sqrt(dist.gm.sigma[min_index,var_idx,var_idx])
    return 5*std



In [3]:
def start_SOGA_smooth(cfg, params_dict={}, pruning=None, Kmax=None, parallel=None):
    """ Invokes SOGA on the root of the CFG object cfg, initializing current_distribution to a Dirac delta centered in zero.
        eps is a dictionary containing 'eps_asgmt' and 'eps_test', the smoothing coefficients for assignments and if branches."""

    # initializes current_dist
    var_list = cfg.ID_list
    data = cfg.data
    n_dim = len(var_list)
    gm = GaussianMix(torch.tensor([[1.]]), torch.zeros((1,n_dim)), EPS*torch.eye(n_dim).reshape(1,n_dim, n_dim))
    init_dist = Dist(var_list, gm)
    cfg.root.set_dist(init_dist)
    
    # initializes visit queue
    exec_queue = [cfg.root]
    
    # executes SOGA on nodes on exec_queue
    while(len(exec_queue)>0):
        SOGAsmooth(exec_queue.pop(0), data, parallel, exec_queue, params_dict)
    
    # returns output distribution
    p, current_dist = merge(cfg.node_list['exit'].list_dist)
    cfg.node_list['exit'].list_dist = []
    return current_dist


def SOGAsmooth(node, data, parallel, exec_queue, params_dict):

    #print('Entering', node)
    #print(node.dist)
    #print('\n')

    if node.type != 'merge' and node.type != 'exit':
        current_dist = node.dist                  # previously copy_dist(node.dist)
        current_p = node.p
        current_trunc = node.trunc
        

    # starts execution
    if node.type == 'entry':
        update_child(node.children[0], node.dist, torch.tensor(1.), None, exec_queue)
            
    
    # if tests saves LBC and calls on children
    if node.type == 'test':
        current_trunc = node.LBC
        if '==' in current_trunc or '!=' in current_trunc:
            current_trunc = smooth_trunc(current_trunc, node)
        for child in node.children:
            update_child(child, node.dist, current_p, current_trunc, exec_queue)
            

    # if loop saves checks the condition and decides which child node must be accessed
    if node.type == 'loop':
        # the first time is accessed set the value of the counter to 0 and converts node.const into a number
        if data[node.idx][0] is None:
            data[node.idx][0] = torch.tensor(0.)
        if type(node.const) is str:
            if '[' in node.const:
                data_name, data_idx = node.const.split('[')
                data_idx = data_idx[:-1]
                # data_idx is a data
                if data_idx in data:
                    data_idx = int(data[data_idx][0])
                # data_idx is a number
                else:
                    data_idx = int(data_idx)
                node.const = torch.tensor(int(data[data_name][data_idx]))
            else:
                node.const = torch.tensor(int(node.const))            
        # successively checks the condition and decides which child node must be accessed
        if data[node.idx][0] < node.const:
            for child in node.children:
                if child.cond == True:
                    update_child(child, node.dist, current_p, current_trunc, exec_queue)
        else:
            data[node.idx][0] = None
            for child in node.children:
                if child.cond == False:
                    update_child(child, node.dist, current_p, current_trunc, exec_queue)
     

    # if state checks wheter cond!=None. If yes, truncates to current_trunc, eventually negating it. In any case applies the rule in expr. Appends the distribution in the next merge node or calls recursively on children. If child is loop node increments its idx.
    if node.type == 'state':
        if node.cond != None and not current_trunc is None:
            if node.cond == False:
                current_trunc = negate(current_trunc) 
            p, current_dist = truncate(current_dist, current_trunc, data, params_dict)     ### see libSOGAtruncate
            current_trunc = None
            current_p = p*current_p
        if current_p > TOL_PROB:
            updated_dist = update_rule(current_dist, node.expr, data, params_dict)         ### see libSOGAupdate
            
            # smoothing
            smooth_flag = check_dist_non_deg(updated_dist)
            if smooth_flag:
                updated_dist = smooth_asgmt(updated_dist, node, data, params_dict)
            current_dist = updated_dist

        # updating child
        child = node.children[0]
        if child.type == 'loop' and not data[child.idx][0] is None:
            data[child.idx][0] += 1
        update_child(child, current_dist, current_p, current_trunc, exec_queue)
        
            
    # if observe truncates to LBC and calls on children
    if node.type == 'observe':
        current_trunc = node.LBC
        #if parallel is not None and parallel >1:
        #    p, current_dist = parallel_truncate(current_dist, current_trunc, data,parallel)
        #else:
        p, current_dist = truncate(current_dist, current_trunc, data, params_dict)                     ### see libSOGAtruncate
        #current_p = current_p*p
        current_trunc = None
        child = node.children[0]
        update_child(child, current_dist, current_p, current_trunc, exec_queue)


    # if merge checks whether all paths have been explored.
    # Either returns or merge distributions and calls on children
    if node.type == 'merge':
        if len(node.list_dist) != len(node.parent):
            return
        else:
            current_p, current_dist = merge(node.list_dist)        ### see libSOGAmerge
            node.list_dist = []
            child = node.children[0]
            update_child(child, current_dist, current_p, None, exec_queue)
                
                
    if node.type == 'exit':
        return
    
    #if node.type == 'prune':
    #    current_dist = prune(current_dist,'classic',node.Kmax)        ### options: 'classic', 'ranking' (see libSOGAmerge)
    #    node.list_dist = []
    #    for child in node.children:
    #        if child.type == 'merge' or child.type == 'exit':
    #            child.list_dist.append((current_p, current_dist))
    #        else:
    #            child.set_dist(copy(current_dist))
    #            child.set_p(current_p)
    #            child.set_trunc(current_trunc)
    #        exec_queue.append(child)

### Test 1: Assigning Constants

In [4]:
text = 'y = gauss(0,1); x = 0;'

text = compile2SOGA_text(text)
cfg = produce_cfg_text(text)
output_dist = start_SOGA(cfg)
output_dist

Dist<['y', 'x'],pi: tensor([[1.]])
mu: tensor([[0., 0.]])
sigma: tensor([[[1., 0.],
         [0., 0.]]])>

In [5]:
smooth_output_dist = start_SOGA_smooth(cfg)
smooth_output_dist

Dist<['y', 'x'],pi: tensor([[1.]])
mu: tensor([[0., 0.]])
sigma: tensor([[[1.0000e+00, 0.0000e+00],
         [0.0000e+00, 1.0000e-04]]])>

### Test 2: Assigning Discrete Distributions

In [6]:
text = 'y = gauss(0,1); x = bern(0.5);'

text = compile2SOGA_text(text)
cfg = produce_cfg_text(text)
output_dist = start_SOGA(cfg)
output_dist

Dist<['y', 'x'],pi: tensor([[0.5000],
        [0.5000]])
mu: tensor([[0., 0.],
        [0., 1.]])
sigma: tensor([[[1., 0.],
         [0., 0.]],

        [[1., 0.],
         [0., 0.]]])>

In [7]:
smooth_output_dist = start_SOGA_smooth(cfg)
smooth_output_dist

Dist<['y', 'x'],pi: tensor([[0.2500],
        [0.2500],
        [0.2500],
        [0.2500]])
mu: tensor([[0., 0.],
        [0., 0.],
        [0., 1.],
        [0., 1.]])
sigma: tensor([[[1.0000e+00, 0.0000e+00],
         [0.0000e+00, 1.0000e-04]],

        [[1.0000e+00, 0.0000e+00],
         [0.0000e+00, 1.0000e-04]],

        [[1.0000e+00, 0.0000e+00],
         [0.0000e+00, 1.0000e-04]],

        [[1.0000e+00, 0.0000e+00],
         [0.0000e+00, 1.0000e-04]]])>

### Test 3: Assigning Constant times Discrete

In [8]:
text = 'y = gauss(0,1); x = 2*gm([0.5, 0.5], [0., 1.], [0., 0.]);'

text = compile2SOGA_text(text)
cfg = produce_cfg_text(text)
output_dist = start_SOGA(cfg)
output_dist


Dist<['y', 'x'],pi: tensor([[0.5000],
        [0.5000]])
mu: tensor([[0., 0.],
        [0., 2.]])
sigma: tensor([[[1., 0.],
         [0., 0.]],

        [[1., 0.],
         [0., 0.]]])>

In [9]:
smooth_output_dist = start_SOGA_smooth(cfg)
smooth_output_dist

Dist<['y', 'x'],pi: tensor([[0.2500],
        [0.2500],
        [0.2500],
        [0.2500]])
mu: tensor([[0., 0.],
        [0., 0.],
        [0., 2.],
        [0., 2.]])
sigma: tensor([[[1.0000e+00, 0.0000e+00],
         [0.0000e+00, 1.0000e-04]],

        [[1.0000e+00, 0.0000e+00],
         [0.0000e+00, 1.0000e-04]],

        [[1.0000e+00, 0.0000e+00],
         [0.0000e+00, 1.0000e-04]],

        [[1.0000e+00, 0.0000e+00],
         [0.0000e+00, 1.0000e-04]]])>

### Test 4: Assigning Other Variables

In [10]:
text = 'y = gauss(0,1); x = y;'

text = compile2SOGA_text(text)
cfg = produce_cfg_text(text)
output_dist = start_SOGA(cfg)
output_dist



Dist<['y', 'x'],pi: tensor([[1.]])
mu: tensor([[0., 0.]])
sigma: tensor([[[1., 1.],
         [1., 1.]]])>

In [11]:
smooth_output_dist = start_SOGA_smooth(cfg)
smooth_output_dist

Dist<['y', 'x'],pi: tensor([[1.]])
mu: tensor([[0., 0.]])
sigma: tensor([[[1.0000, 1.0000],
         [1.0000, 1.0001]]])>

In [12]:
cfg.node_list

{'entry': EntryNode<>,
 'state0': StateNode<state0,None,y=gm([1.0],[0.000000],[1.000000])>,
 'state1': StateNode<state1,None,x=y,smoothed:x=y+ gm([1.], [0.], [0.01])>,
 'exit': ExitNode<>}

### Test 5: If Clauses depending on Extreme Discrete Values

In [13]:
text = 'x = bern(0.5); if x == 1 { x = x + 1; } else { x = x - 1; } end if;'

text=compile2SOGA_text(text)
cfg = produce_cfg_text(text)
#output_dist = start_SOGA(cfg)

#print('Before smoothing')
cfg.node_list


{'entry': EntryNode<>,
 'state0': StateNode<state0,None,x=gm([0.500000,0.500000],[0.0,1.0],[0.0,0.0])>,
 'test0': TestNode<test0,x==1>,
 'state1': StateNode<state1,True,x=x+1>,
 'state2': StateNode<state2,False,x=x-1>,
 'merge0': MergeNode<merge0>,
 'exit': ExitNode<>}

In [14]:
smooth_output_dist = start_SOGA_smooth(cfg)
smooth_output_dist


truncate x > 0.97 and x < 1.03
Calling and_func
Returning tensor(0.4987) Dist<['x'],pi: tensor([[0.5000],
        [0.5000]])
mu: tensor([[1.],
        [1.]])
sigma: tensor([[[9.7334e-05]],

        [[9.7334e-05]]])>
truncate x <= 0.97 or x >= 1.03


TypeError: 'NoneType' object is not callable

In [ ]:
cfg.node_list

{'entry': EntryNode<>,
 'state0': StateNode<state0,None,x=gm([0.500000,0.500000],[0.0,1.0],[0.0,0.0]),smoothed:x=gm([0.500000,0.500000],[0.0,1.0],[0.0,0.0])+ gm([1.], [0.], [0.01])>,
 'test0': TestNode<test0,x==1>,
 'state1': StateNode<state1,True,x=x+1>,
 'state2': StateNode<state2,False,x=x-1>,
 'merge0': MergeNode<merge0>,
 'exit': ExitNode<>}

### Test 5: If Clauses depending on Discrete Variables + Double Degenerate update

In [ ]:
text = 'x = bern(0.5); if x == 1 { y = 1; } else { y = -1; } end if;'

compiledFile=compile2SOGA_text(text)
cfg = produce_cfg_text(compiledFile)
#output_dist = start_SOGA(cfg)

#print('Before smoothing')
#print(cfg.node_list)


In [ ]:
def start_SOGA_smoother(compiledFile):
    """ Checks every node in the cfg to check for instructions causing degeneracy.
        Return the cfg of the smoothed program. """
    
    # initializes a dictionary for smoothed variables
    smoothed_vars = {}

    smooth_flag = True

    while smooth_flag:

        cfg = produce_cfg_text(compiledFile)

        # initializes current_dist
        var_list = cfg.ID_list
        data = cfg.data
        n_dim = len(var_list)
        gm = GaussianMix(torch.tensor([[1.]]), torch.zeros((1,n_dim)), EPS*torch.eye(n_dim).reshape(1,n_dim, n_dim))
        init_dist = Dist(var_list, gm)
        cfg.root.set_dist(init_dist)
        params_dict = {}
    
        # initializes visit queue
        exec_queue = [cfg.root]
    
        # executes SOGA on nodes on exec_queue
        while(len(exec_queue)>0):
        
            current_eps = SMOOTH_EPS
            current_node = exec_queue.pop(0)

            # computes the new distribution in SOGA and checks whether degeneracies have arised
            current_dist, current_p, current_trunc = SOGA_smoother(current_node, data, exec_queue, params_dict)
            smooth_flag = check_degeneracy(current_dist, current_trunc)

            # if degeneracy is due to an assignment, we need to add Gaussian noise 
            while smooth_flag == 'dist':
                print('degeneracy in distribution detected in node ', current_node)
                current_node.expr = current_node.expr + ' + gm([1.], [0.], [{}])'.format(current_eps)
                check_smoothed_var(current_node, current_dist, current_eps, smoothed_vars)
                print(smoothed_vars)
                current_eps = current_eps + SMOOTH_DELTA
                # computes the SOGA semantics with the correction and check degeneracy again
                current_dist, current_p, current_trunc = SOGA_smoother(current_node, data, exec_queue, params_dict)
                smooth_flag = check_degeneracy(current_dist, current_trunc)
            # when degeneracy is solved updates the children
            for child in current_node.children:
                update_child(child, current_dist, current_p, current_trunc, exec_queue)
                
            # if degeneracy is due to a truncation, we need to correct the program
            if smooth_flag == 'trunc':
                print('degeneracy in truncation detected in node ', current_node, ' for ', current_trunc)
                new_program = correct_degeneracy_trunc(current_node, current_trunc, smoothed_vars)
                break

        if new_program:
            compiledFile = new_program
        else:
            smooth_flag = False


### Test 6: If Clauses depending on intermediate values

In [ ]:
text = 'x = gm([0.33, 0.33, 0.34], [0., 1. , 2.], [0., 0., 0.]); if x == 1 { y = 1; } else { y = 0; } end if;'

compiledFile=compile2SOGA_text(text)
cfg = produce_cfg_text(compiledFile)
#output_dist = start_SOGA(cfg)

#print('Before smoothing')
cfg.node_list

{'entry': EntryNode<>,
 'state0': StateNode<state0,None,x=gm([0.33,0.33,0.34],[0.,1.,2.],[0.,0.,0.])>,
 'test0': TestNode<test0,x==1>,
 'state1': StateNode<state1,True,y=1>,
 'state2': StateNode<state2,False,y=0>,
 'merge0': MergeNode<merge0>,
 'exit': ExitNode<>}

In [ ]:
def correct_degeneracy_trunc(node, trunc, smoothed_vars):
    """ 
        Correct degeneracy due to the presence of branches with conditions var == val.
        It changes the conditions into one or more ifs with conditios var >= val or var <= val.
    """

    # parses the truncation, saving target variable and value
    var, val = trunc.split('==')
    var = var.strip()
    val = int(val)

    # Updates the truncation according to the values of the smoothed variable
    if var in smoothed_vars.keys():
        
        val_idx = smoothed_vars[var]['orig_supp'].index(val)
        orig_supp = smoothed_vars[var]['orig_supp']

        # branching on the smallest possible value of the variable
        if val_idx == 0:
            if len(smoothed_vars[var]['orig_supp']) >= 1:
                next_val = smoothed_vars[var]['orig_supp'][1]
                delta = (next_val - val)/2
            else:
                delta = 5*smoothed_vars[var]['std']
                new_condition = var + '<' + str(val + delta)
        # branching on the largest possible value of the variable
        elif val_idx == len(smoothed_vars[var]['orig_supp']) - 1:
            prev_val = smoothed_vars[var]['orig_supp'][val_idx - 1]
            delta = (val - prev_val)/2
            new_condition = var + '>' + str(val - delta)
        # branching on intermediate values requires adding nested ifs
        else:
            prev_val = smoothed_vars[var]['orig_supp'][val_idx - 1]
            next_val = smoothed_vars[var]['orig_supp'][1]
            delta_prev = (val - prev_val)/2
            delta_next = (next_val - val)/2
            for child in node.children:
                if child.cond == True:
                    then_prog = extract_subprogram(child)
                elif child.cond == False:
                    else_prog = extract_subprogram(child)
            inner_if = create_if(var + '<' + str(val + delta_next), then_prog, else_prog)
            outer_if = create_if(var + '>' + str(val - delta_prev), inner_if, else_prog)
                      
    else: 
        print('Invalid if conditions: Equality constraint with continuous variable ', current_trunc)
        raise RuntimeError

In [ ]:
cfg = create_if('x > 0', 'y = 1;', 'y = 0;')
cfg.node_list

{'entry': EntryNode<>,
 'test0': TestNode<test0,x>0>,
 'state0': StateNode<state0,True,y=1>,
 'state1': StateNode<state1,False,y=0>,
 'merge0': MergeNode<merge0>,
 'exit': ExitNode<>}

In [ ]:
extract_subprogram(cfg.root)

EntryNode<>  []
TestNode<test0,x>0>  []
StateNode<state0,True,y=1> if x>0 { [TestNode<test0,x>0>]
MergeNode<merge0> if x>0 {y=1; [TestNode<test0,x>0>]
StateNode<state1,False,y=0> if x>0 {y=1; } else { []
MergeNode<merge0> if x>0 {y=1; } else {y=0; []


'if x>0 {y=1; } else {y=0; } end if;'

In [ ]:
def create_if(cond, then_prog, else_prog):
    prog = 'if {} {{ {} }} else {{ {} }} end if;'.format(cond, then_prog, else_prog)
    return prog



#def create_partial_cfg(cfg):
#    """ Removes entry and exit nodes from a cfg and returns the new root"""
    # setting new_root
#    root = cfg.node_list['entry'].children[0]
#    root.parent = None
    # disconnecting exit node 
    #if len(cfg.node_list['exit'].parent) == 1:
    #    cfg.node_list['exit'].parent[0].children = []
    #else:
    #    # if the exit node has more than one parent, we need to add a merge node
    #    node = MergeNode('merge{}'.format(self.n_merge))
    #    cfg.n_merge += 1
    #    node.parent, _ = cfg.node_list['exit'].parent
    #    for parent in node.parent:
    #        parent.children.append(node)
    #    cfg.node_list[node.name] = node

#    del cfg.node_list['entry']
    #del cfg.node_list['exit']

#def extract_text(cfg):
#    """ Extracts the text from a cfg"""
#    # replaces the root with a skip node
#    skip_node = StateNode('skip')
#    root = cfg.node_list['entry'].children[0]
#    root.parent = skip_node
#    # replaces the exit node with a merge
#    node = MergeNode('merge{}'.format(cfg.n_merge))
#    cfg.n_merge += 1
#    node.parent, _ = cfg.node_list['exit'].parent
#    for parent in node.parent:
#        parent.children.append(node)
#    cfg.node_list[node.name] = node
#    #deletes older entry and exit nodes
#    del cfg.node_list['entry']
#    del cfg.node_list['exit']
#    return extract_subprogram(skip_node)

def extract_subprogram(node, subprogram='', stack=[]):
    """ Extracts the subprogram starting from a state node and returns it as a string"""
    
    print(node, subprogram, stack)

    if node.type == 'entry':
        child = node.children[0]
        subprogram = extract_subprogram(child, subprogram, stack)

    if node.type == 'state':
        subprogram += node.expr + ';'
        child = node.children[0]
        subprogram = extract_subprogram(child, subprogram, stack)
    
    if node.type == 'test':
        subprogram += 'if ' + node.LBC + ' {'
        stack.append(node)
        for child in node.children:
            if child.cond is True:
                subprogram = extract_subprogram(child, subprogram, stack)
        
    if node.type == 'merge':
        if len(stack) > 0:
            subprogram += ' } else {'
            test = stack.pop()
            for child in test.children:
                if child.cond is False:
                    subprogram = extract_subprogram(child, subprogram, stack)
                    subprogram += ' } end if;'
    
    return subprogram

In [ ]:
text = 'if x > 0.5 { if x < 1.5 { y = gauss(1,1); } else { y = gauss(0,1); } end if; } else { y = gauss(0,1); } end if;'

compiledFile=compile2SOGA_text(text)
cfg = produce_cfg_text(compiledFile)

cfg.node_list


{'entry': EntryNode<>,
 'test0': TestNode<test0,x>0.5>,
 'state0': StateNode<state0,True,skip>,
 'test1': TestNode<test1,x<1.5>,
 'state1': StateNode<state1,True,y=gm([1.0],[1.000000],[1.000000])>,
 'state2': StateNode<state2,False,y=gm([1.0],[0.000000],[1.000000])>,
 'merge0': MergeNode<merge0>,
 'state3': StateNode<state3,False,y=gm([1.0],[0.000000],[1.000000])>,
 'merge1': MergeNode<merge1>,
 'exit': ExitNode<>}

In [ ]:
subprogram = extract_subprogram(cfg.node_list['state0'])
new_cfg = produce_cfg_text(subprogram)


StateNode<state0,True,skip>  []
TestNode<test1,x<1.5> skip; []
StateNode<state1,True,y=gm([1.0],[1.000000],[1.000000])> skip;if x<1.5 { [TestNode<test1,x<1.5>]
MergeNode<merge0> skip;if x<1.5 {y=gm([1.0],[1.000000],[1.000000]); [TestNode<test1,x<1.5>]
StateNode<state2,False,y=gm([1.0],[0.000000],[1.000000])> skip;if x<1.5 {y=gm([1.0],[1.000000],[1.000000]); } else { []
MergeNode<merge0> skip;if x<1.5 {y=gm([1.0],[1.000000],[1.000000]); } else {y=gm([1.0],[0.000000],[1.000000]); []


In [ ]:
subprogram

'skip;if x<1.5 {y=gm([1.0],[1.000000],[1.000000]); } else {y=gm([1.0],[0.000000],[1.000000]); } end if;'